This notebook will be mainly used for the capstone project.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
import requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.7.1-py36_1 --> 4.8.2-py36_0 conda-forge


beautifulsoup4-4.8.2 | 157 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(req.text, 'lxml')

# print(soup.prettify())

In [5]:
table = soup.find('table',class_='wikitable sortable')

print(table)


 

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [6]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [7]:
for row in table.find_all('tr'):
    postalcode = ''
    borough = ''
    neighborhood = ''
    
    for column in row.find_all('td'):
        if postalcode == '' :
            postalcode = column.text
        else :
            if borough == '':
                borough = column.text
            else :
                if neighborhood == '':
                    neighborhood = column.text
        
        # print(column.text)
        
    neighborhoods = neighborhoods.append({'PostalCode': postalcode,
                                            'Borough': borough,
                                          'Neighborhood': neighborhood}, ignore_index=True)
        
    # print()
    
    
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,,,,NaN,NaN
1,M1A,Not assigned,Not assigned\n,NaN,NaN
2,M2A,Not assigned,Not assigned\n,NaN,NaN
3,M3A,North York,Parkwoods\n,NaN,NaN
4,M4A,North York,Victoria Village\n,NaN,NaN


In [8]:
neighborhoods['Borough'].replace('', np.nan, inplace=True)

neighborhoods['Borough'].replace('Not assigned', np.nan, inplace=True)

neighborhoods.dropna(subset=['Borough'], inplace=True)

neighborhoods['Neighborhood'].replace('Not assigned\n', neighborhoods['Borough'], inplace=True)

neighborhoods = neighborhoods.replace('\n','', regex=True)

neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,NaN,NaN
4,M4A,North York,Victoria Village,NaN,NaN
5,M5A,Downtown Toronto,Harbourfront,NaN,NaN
6,M6A,North York,Lawrence Heights,NaN,NaN
7,M6A,North York,Lawrence Manor,NaN,NaN
8,M7A,Downtown Toronto,Queen's Park,NaN,NaN
10,M9A,Queen's Park,Queen's Park,NaN,NaN
11,M1B,Scarborough,Rouge,NaN,NaN
12,M1B,Scarborough,Malvern,NaN,NaN
14,M3B,North York,Don Mills North,NaN,NaN


In [9]:
neighborhoods.set_index(['PostalCode','Borough'],inplace=True)

#strJoin = lambda x:",".join(x.astype(str))     
neighborhoods = neighborhoods.groupby(level=['PostalCode','Borough']).agg({'Neighborhood':', '.join})

neighborhoods

,,Neighborhood
PostalCode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"


In [10]:
print(neighborhoods.shape)

(103, 1)
